In [296]:
#declare dependencis
 
from flask_pymongo import PyMongo
import pandas as pd
import datetime
import pprint as pp
import pymongo
# Import Scrape libraries
import requests
import urllib.request
import time
from splinter import Browser
from bs4 import BeautifulSoup as bs
from urllib.parse import urljoin
import itertools

## Definitions 

In [297]:
def getridoftoprow(df):
    '''Nifty routine to drop top row of double headed table'''
    dft=df.T 
    dft=dft.drop(dft.columns[[0]], axis=1)
    df=dft.T
    return df

def geturlresponse(url):
    '''Get time out, used more than once'''
    #This will get a response within ten seconds or just move on to the next table
    try:
        response= requests.get(url, timeout=10 )
    except requests.exceptions.Timeout:
        pass   
    return response

# Initialize the Database with Mongo, 

In [298]:


conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
# declare database
db = client.water_db  # implicitly declared mars DB
collection = db.water_db


## I decided to not use the chromedriver as it is problematic and slowish, and not necessary for this task

In [299]:
#executable path to driver
# executable_path = {'executable_path': 'chromedriver.exe'}
# browser = Browser('chrome', **executable_path, headless=False)

#Chromedriver really kinds of sucks, it is slow

### This is the EPA table for recommened levels of a list of chemicals for Aquatic Life, not drinking water.  So this is about the fish.

In [300]:
epa_url = 'https://www.epa.gov/wqc/national-recommended-water-quality-criteria-aquatic-life-criteria-table'
#browser.visit(epa_url)  #not using this, it will open up Chrome to the selected web page 

#This will get a response within ten seconds or just move on to the next table
#If it returns 200 worko!
geturlresponse(epa_url)

<Response [200]>

## This table shows pollutant levels and has a concatenated CAS number, the definitive identifier of chemicals.  There is a list of over eight hundred thousand maintained by the EPA.  Chemicals are often referred to by different names so it was imperative to have a unique identifier that chemists use.  The CAS number, as represented here is not properly stuctured, but we can still use it.


In [301]:

#Use Pandas to read all the tables on the page.  
epa_tables = pd.read_html(epa_url) 
getridoftoprow(epa_tables[0]).head()
#df_1.head()

,Pollutant (P = Priority Pollutant),CAS Number,Freshwater CMC1 (acute) (µg/L),Freshwater CCC2 (chronic) (µg/L),Saltwater CMC1 (acute) (µg/L),Saltwater CCC2 (chronic) (µg/L),Publication Year,Notes
1,Aesthetic Qualities,—,—,—,—,—,1986,"See Quality Criteria for Water, 1986 (""Gold Bo..."
2,Aldrin (P),309002,3.0,—,1.3,—,1980,These criteria are based on the 1980 criteria ...
3,Alkalinity,—,—,20000,—,—,1986,The CCC of 20mg/L is a minimum value except wh...
4,alpha-Endosulfan (P),959988,0.22,0.056,0.034,0.0087,1980,These criteria are based on the 1980 criteria ...
5,Aluminum pH 5.0 - 10.5,7429905,--,--,—,—,2018,The criteria is based on the water chemistry d...


In [302]:
human_health = 'https://www.epa.gov/wqc/national-recommended-water-quality-criteria-human-health-criteria-table'
#browser.visit(human_health)
geturlresponse(human_health)

<Response [200]>

In [303]:
human_health_df = pd.read_html(human_health)

 
human_health_df= getridoftoprow(human_health_df[0]) 
human_health_df.loc[:,'Usage']='Water Testing' # add a new column
human_health_df.columns=['Pollutant','CAS','Human Health for the consumption of Water + Organism (µg/L)','Human Health for the consumption of Organism Only (µg/L)','Publication Year','Notes','Usage']
 
human_health_df.head()

,Pollutant,CAS,Human Health for the consumption of Water + Organism (µg/L),Human Health for the consumption of Organism Only (µg/L),Publication Year,Notes,Usage
1,Acrolein (P),107028,3,400,2015,NaN,Water Testing
2,Acrylonitrile (P),107131,0.061,7.0,2015,This criterion is based on carcinogenicity of ...,Water Testing
3,Aldrin (P),309002,0.00000077,0.00000077,2015,This criterion is based on carcinogenicity of ...,Water Testing
4,alpha-Hexachlorocyclohexane (HCH) (P),319846,0.00036,0.00039,2015,This criterion is based on carcinogenicity of ...,Water Testing
5,alpha-Endosulfan (P),959988,20,30,2015,NaN,Water Testing


## Frac list of chemicals, identified by CAS number in a padded form with leading zeros.  This is almost certainly the best way to represent CAS numbers, but we are going to undo it with Pandas so that it has the same format as the Water Pollutant table .

In [304]:
frac = 'https://fracfocus.org/chemical-use/what-chemicals-are-used'
geturlresponse(frac)

<Response [200]>

In [305]:
frac_df = pd.read_html(frac)
hydraulic = frac_df[0]
hydraulic.columns = hydraulic.iloc[0]
hydraulic = getridoftoprow(hydraulic)
hydraulic_pure = hydraulic.dropna()

hydraulic_pure.head()

,Chemical Name,CAS,Chemical Purpose,Product Function
1,Hydrochloric Acid,007647-01-0,Helps dissolve minerals and initiate cracks i...,Acid
3,Glutaraldehyde,000111-30-8,Eliminates bacteria in the water that produce...,Biocide
4,Quaternary Ammonium Chloride,012125-02-9,Eliminates bacteria in the water that produce...,Biocide
5,Quaternary Ammonium Chloride,061789-71-1,Eliminates bacteria in the water that produce...,Biocide
6,Tetrakis Hydroxymethyl-Phosphonium Sulfate,055566-30-8,Eliminates bacteria in the water that produce...,Biocide


## This table is processed and then a new Column with a massaged CAS number is generated. Also, an additional columns is added in, Usage.  

In [306]:
hydraulic_pure.loc[:,1]=(hydraulic_pure['CAS']).str.lstrip('0').replace('-','',regex=True)  #
hydraulic_pure.loc[:,'Usage']='Fracking' # add a new column
hydraulic_pure.columns=['Chemical Name','Original CAS','Chemical Purpose','Product Function','CAS','Usage']

hydraulic_pure.head()

,Chemical Name,Original CAS,Chemical Purpose,Product Function,CAS,Usage
1,Hydrochloric Acid,007647-01-0,Helps dissolve minerals and initiate cracks i...,Acid,7647010,Fracking
3,Glutaraldehyde,000111-30-8,Eliminates bacteria in the water that produce...,Biocide,111308,Fracking
4,Quaternary Ammonium Chloride,012125-02-9,Eliminates bacteria in the water that produce...,Biocide,12125029,Fracking
5,Quaternary Ammonium Chloride,061789-71-1,Eliminates bacteria in the water that produce...,Biocide,61789711,Fracking
6,Tetrakis Hydroxymethyl-Phosphonium Sulfate,055566-30-8,Eliminates bacteria in the water that produce...,Biocide,55566308,Fracking


# Now we have two tables, hydraulic_pure and human_health_df which have a common identifier, CAS.  We can do some joining 
## Executing an inner join yields no common chemicals, which is kind of disturbing.  What is signifies is that none of the chemicals used in fracking are tested for in drinking water, not even methanol, CH4.  Of course things can vary, but a no set in this case is signifcant.  So, the generated set here will be an outer join that assures that we have all data merged together, and that is what we will send to MONGODB.

In [307]:
#reference with shorter names
dfh=human_health_df
dff=hydraulic_pure 
df=pd.merge(dfh, dff, on='CAS', how='outer')
df.head()

,Pollutant,CAS,Human Health for the consumption of Water + Organism (µg/L),Human Health for the consumption of Organism Only (µg/L),Publication Year,Notes,Usage_x,Chemical Name,Original CAS,Chemical Purpose,Product Function,Usage_y
0,Acrolein (P),107028,3,400,2015,NaN,Water Testing,NaN,NaN,NaN,NaN,NaN
1,Acrylonitrile (P),107131,0.061,7.0,2015,This criterion is based on carcinogenicity of ...,Water Testing,NaN,NaN,NaN,NaN,NaN
2,Aldrin (P),309002,0.00000077,0.00000077,2015,This criterion is based on carcinogenicity of ...,Water Testing,NaN,NaN,NaN,NaN,NaN
3,alpha-Hexachlorocyclohexane (HCH) (P),319846,0.00036,0.00039,2015,This criterion is based on carcinogenicity of ...,Water Testing,NaN,NaN,NaN,NaN,NaN
4,alpha-Endosulfan (P),959988,20,30,2015,NaN,Water Testing,NaN,NaN,NaN,NaN,NaN


# Step 3 - Passing the data to a database.  This should be easy...let's hope so.  Above, I already have a named collection,in my water_db database.

In [308]:
from pymongo import MongoClient
import pandas as pd
 

data = df.to_dict(orient='records')  # Here's our added param..
collection.drop() #clean things up before adding
collection.insert_many(data)

# CONCLUSION and COMMENT
### This process is really straight forward and for us, was made more complicated by the fact that we also had a complex homework assignment to finish, which I, Kevin, gave priority. We could have chosen a topic a bit more straight forward, but we all agreed that whatever we do should have the possiblity of an added value.  In this case, the added value turned out to be that in our preliminary analysis there does not seem to be any testing of commonly used fracking chemicals in drinking water. That should be of considerable interest to people who have a water source above a frack zone.

### Also, the use of MONGODB for this type of project is awesome.  All we had to do at the end was there lines of code, that could have been two...essentially one line of code if we don't clean house before committing. 

## That is efficient.

#### In my opinon, this was a good project, but I think it should have been scheduled a bit differently.
